In [ ]:
import pathlib

from astropy.convolution import Gaussian2DKernel, convolve
import astropy.coordinates as coord
from astropy.io import ascii, fits
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy.stats import binned_statistic, binned_statistic_2d
from IPython.display import HTML
from astropy.stats import median_absolute_deviation as MAD
from matplotlib.animation import FuncAnimation

# gala
import gala.coordinates as gc
import gala.dynamics as gd
import gala.potential as gp
from gala.units import galactic

from pyia import GaiaData
from cmastro import cmaps

# Make parent data file: Gaia EDR3 x actions x 2MASS

TODO: recompute actions in my PW2021 model

In [ ]:
_cache_file = pathlib.Path('../data/edr3-2mass-actions.fits').resolve()
_cache_file.parent.mkdir(exist_ok=True)

if not _cache_file.exists():
    edr3 = at.Table.read('/Users/apricewhelan/data/GaiaEDR3/edr3-rv-good-plx-result.fits')
    edr3 = edr3[edr3['parallax_over_error'] > 5]

    aafs = at.Table.read('/Users/apricewhelan/data/GaiaEDR3/edr3-rv-good-plx-result-actions.fits')
    tmass = at.Table.read('/Users/apricewhelan/data/GaiaEDR3/edr3-rv-good-plx-2mass-xm.fits.gz')

    data = at.join(edr3, aafs, keys='source_id')
    data = at.join(data, tmass, keys='source_id', join_type='left')
    data = at.unique(data, keys='source_id')
    del edr3, aafs, tmass
    
    from dustmaps.bayestar import BayestarQuery
    q = BayestarQuery()
    ebv = q.query(c, mode='best')
    data['EBV_bayestar19'] = ebv
    
    keep_cols = [
        'source_id',
        'ra',
        'ra_error',
        'dec',
        'dec_error',
        'parallax',
        'parallax_error',
        'parallax_over_error',
        'pm',
        'pmra',
        'pmra_error',
        'pmdec',
        'pmdec_error',
        'ra_dec_corr',
        'ra_parallax_corr',
        'ra_pmra_corr',
        'ra_pmdec_corr',
        'dec_parallax_corr',
        'dec_pmra_corr',
        'dec_pmdec_corr',
        'parallax_pmra_corr',
        'parallax_pmdec_corr',
        'pmra_pmdec_corr',
        'ruwe',
        'phot_g_mean_flux_over_error',
        'phot_g_mean_mag',
        'phot_bp_mean_flux_over_error',
        'phot_bp_mean_mag',
        'phot_rp_mean_flux_over_error',
        'phot_rp_mean_mag',
        'dr2_radial_velocity',
        'dr2_radial_velocity_error',
        'E',
        'L_x',
        'L_y',
        'L_z',
        'ecc',
        'r_apo',
        'r_per',
        'z_max',
        'J_R',
        'J_phi',
        'J_z',
        'theta_R',
        'theta_phi',
        'theta_z',
        'Om_R',
        'Om_phi',
        'Om_z',
        'j_m',
        'j_msigcom',
        'h_m',
        'h_msigcom',
        'ks_m',
        'ks_msigcom',
        'EBV_bayestar19'
    ]
    
    data[keep_cols].write(_cache_file)
    
else:
    data = at.Table.read(_cache_file)

# CMD selections based on 2MASS photometry:

In [ ]:
iso = at.Table.read('/Users/apricewhelan/data/Isochrones/old_MIST/FeH_-0.2_iso.fits')
iso = iso[iso['log10_isochrone_age_yr'] == 7.6]
iso = iso[iso['phase'] <= 1]

In [ ]:
g = GaiaData(data)

In [ ]:
AJ = 0.72 * g.EBV_bayestar19
AH = 0.46 * g.EBV_bayestar19
AKs = 0.306 * g.EBV_bayestar19

In [ ]:
cmd_paths = {}

cmd_paths['ms'] = mpl.path.Path([
    [-0.2, 2],
    [0.1, -1],
    [0.4, 0],
    [0.5, 3.5],
    [0.7, 4.1],
    [0.7, 5.2],
    [0, 3.7]
])

cmd_paths['rc'] = mpl.path.Path([
    [0.5, -2],
    [0.85, -1.8],
    [0.85, -1.1],
    [0.5, -1.45],
])

cmd_paths['rgb'] = mpl.path.Path([
    [0.45, -2.3],
    [0.5, 0], 
    [0.7, 2.3], 
    [1.25, -5], 
    [0.8, -5]
])

# Young main sequence
cmd_paths['yms'] = mpl.path.Path([
    [-0.4, 2],
    [-0.5, 0],
    [0, -2],
    [0.25, 0],
    [0, 3]
])

# Very young main sequence
# cmd_paths['vyms'] = mpl.path.Path([
#     [-0.4, 2],
#     [-0.5, 0],
#     [0, -2],
#     [0.1, 0],
#     [0, 1.8]
# ])

In [ ]:
JMK = g.j_m - g.ks_m - (AJ - AKs)
MH = g.h_m - g.distmod.value - AH
X = np.stack((JMK, MH)).T

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.hist2d(JMK, MH, 
          bins=(np.linspace(-0.5, 2., 333), 
                np.linspace(-6, 8, 333)),
          norm=mpl.colors.LogNorm(), 
          cmap='Greys',
          rasterized=True)

for k, path in cmd_paths.items():
    ax.plot(path.vertices[:, 0], path.vertices[:, 1])

ax.set_xlim(-0.5, 2.)
ax.set_ylim(8, -6)

ax.set_xlabel('$(J-K)_0$')
ax.set_ylabel('$M_{H,0}$')

In [ ]:
cmd_masks = {}
for name, cmd_path in cmd_paths.items():
    _mask = cmd_path.contains_points(X)
    cmd_masks[f'{name}_cmd_mask'] = _mask
    print(f"{name: <5}: {_mask.sum()}")
cmd_masks = at.Table(cmd_masks)

In [ ]:
cmd_masks.write('../data/cmd-masks.fits', overwrite=True)

# Old: Gaia CMD selection

Gaia:

In [ ]:
BPRP = g.phot_bp_mean_mag - g.phot_rp_mean_mag
MG = g.phot_g_mean_mag - g.distmod
# X = np.stack((BPRP.value, MG.value)).T

# fig, ax = plt.subplots(1, 1, figsize=(6, 6))
# ax.hist2d(BPRP.value, MG.value, 
#           bins=(np.linspace(0, 3., 256), 
#                 np.linspace(-4, 10, 256)),
#           norm=mpl.colors.LogNorm())

# nodes = np.array([
#     [2, 8],
#     [1.15, 5.6],
#     [0.9, 3.5],
#     [0.7, 0.5],
#     [0.45, 0.5],
#     [0.1, 1.7],
#     [0.45, 3.5],
#     [0.7, 5.35],
#     [1, 6.5],
#     [2, 9.3]])
# ax.plot(nodes[:, 0], nodes[:, 1], 
#         marker='o', color='tab:green')
# ms_cmd_path = mpl.path.Path(nodes)

# nodes = np.array([
#     [1, 1],
#     [3, 4],
#     [3, 2.6],
#     [1, -0.3]
# ])
# ax.plot(nodes[:, 0], nodes[:, 1], 
#         marker='o', color='tab:green')
# rc_cmd_path = mpl.path.Path(nodes)

# ax.set_xlim(0, 3.)
# ax.set_ylim(10, -4)